In [1]:
import numpy as np
import pickle
import keras.backend as K

from keras.layers import  GlobalAveragePooling1D, Input, Activation, MaxPooling1D, BatchNormalization, Dense, Dropout, Conv1D,GlobalMaxPooling1D
from keras.layers import GRU,AveragePooling1D,CuDNNGRU
from keras.layers.merge import Concatenate
from keras.models import Model 
from keras.callbacks import EarlyStopping,ModelCheckpoint


import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
import os


os.environ["CUDA_VISIBLE_DEVICES"] = "0"

config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)

KTF.set_session(sess)

Using TensorFlow backend.
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xhh/anaconda3/envs/py36/lib/p

In [2]:
import numpy as np
alphabet = np.array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
                     'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'])

def label_sequence(line, MAX_SEQ_LEN, smi_ch_ind):
	X = np.zeros(MAX_SEQ_LEN)

	for i, ch in enumerate(line[:MAX_SEQ_LEN]):
		X[i] = smi_ch_ind[ch]

	return X #.tolist()

def letter_one_hot(aa):
    one_hot = np.zeros(20)
    for idx, letter in enumerate(alphabet):
        if aa == letter:
            one_hot[idx] = 1
            return one_hot


# Convert an entire protein to one-hot representation.
def protein_one_hot(protein_sequence, MAX_SEQ_LEN):
    #  Remove non-specific AA codes (very few are actually present in this dataset)
    protein_sequence = protein_sequence.replace('B', '')
    protein_sequence = protein_sequence.replace('J', '')
    protein_sequence = protein_sequence.replace('O', '')
    protein_sequence = protein_sequence.replace('U', '')
    protein_sequence = protein_sequence.replace('X', '')
    protein_sequence = protein_sequence.replace('Z', '')
    one_hot_seq = np.zeros( (MAX_SEQ_LEN, 20))
    for idx, aa in enumerate(protein_sequence[:MAX_SEQ_LEN]):
        one_hot_seq[idx, :] = letter_one_hot(aa)
    return one_hot_seq


In [3]:
import keras
feature_len = 768
max_go_len = 256
max_seq_len = 1000

from six.moves import cPickle as pickle #for performance

 
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di



class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,  ppi_pair_file, batch_size=128):
        'Initialization' 
        self.batch_size = batch_size
        self.ppi_pair_file = ppi_pair_file
         
        self.max_seqlen = max_seq_len
        self.max_golen = max_go_len
         
        self.protein2seq = load_dict('SPprot2seq.pkl')
        self.read_ppi()
        self.protein2onehot = {}
        self.onehot_seqs()
         
        self.on_epoch_end()
    
    def read_ppi(self):
        with open(self.ppi_pair_file, 'r') as f:
            self.ppi_pairs  =  f.readlines()
    
    def onehot_seqs(self):
        for key, value in self.protein2seq.items():
            self.protein2onehot[key] =  protein_one_hot(value, self.max_seqlen) 
            
    
   

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.ppi_pairs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.ppi_pairs))
         
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

         
        X_seq1 = np.empty((self.batch_size, self.max_seqlen,20))
        X_seq2 = np.empty((self.batch_size, self.max_seqlen,20))
        y = np.empty((self.batch_size))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            ppi_pair = self.ppi_pairs[ID]
            p1, p2, label = ppi_pair.rstrip().split('\t')
            if label == '+':
                y[i] = 1
            else:
                y[i] = 0
            X_seq1[i] =  self.protein2onehot[p1]
            X_seq2[i] =  self.protein2onehot[p2]
             
            
            
        return [  X_seq1, X_seq2] ,  y



    def all_data(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

         
        X_seq1 = np.empty((len(list_IDs_temp), self.max_seqlen,20))

         
        X_seq2 = np.empty((len(list_IDs_temp), self.max_seqlen,20))
        y = np.empty((len(list_IDs_temp)))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            ppi_pair = self.ppi_pairs[ID]
            p1, p2, label = ppi_pair.rstrip().split('\t')
            if label == '+':
                y[i] = 1
            else:
                y[i] = 0
            X_seq1[i] =  self.protein2onehot[p1]
            X_seq2[i] =  self.protein2onehot[p2]
           
        return [  X_seq1, X_seq2] ,  y

In [4]:
from keras import backend as K, initializers, regularizers, constraints
from keras.engine.topology import Layer


def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        # todo: check that this is correct
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)


class Attention(Layer):
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True,
                 return_attention=False,
                 **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Note: The layer has been tested with Keras 1.x
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2 - Get the attention scores
            hidden = LSTM(64, return_sequences=True)(words)
            sentence, word_scores = Attention(return_attention=True)(hidden)
        """
        self.supports_masking = True
        self.return_attention = return_attention
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        eij = dot_product(x, self.W)

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        weighted_input = x * K.expand_dims(a)

        result = K.sum(weighted_input, axis=1)

        if self.return_attention:
            return [result, a]
        return result

    def compute_output_shape(self, input_shape):
        if self.return_attention:
            return [(input_shape[0], input_shape[-1]),
                    (input_shape[0], input_shape[1])]
        else:
            return input_shape[0], input_shape[-1]

In [5]:
from keras.layers import   Embedding
from keras.layers import  GRU, Bidirectional, CuDNNGRU, Lambda, Flatten
from keras.layers.merge import concatenate
def inception_block(input_tensor, output_size):
    """"""
    con1d_filters = int(output_size/4)
    y = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x1 = Conv1D(con1d_filters, 5, activation="relu", padding='same')(y)

    y = Conv1D(con1d_filters, 1, activation="relu", padding='valid')(input_tensor)
    x2 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(y)

    x3 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x4 = Conv1D(con1d_filters, 1, activation="relu", padding='same')(input_tensor)

    y = Concatenate()([x1, x2, x3, x4])
#     y = MaxPooling1D(2)(mix0)
    # y = AveragePooling1D()(mix0)
#     y = BatchNormalization()(y)

    return y

def create_share_model():
    con_filters = 128
    X_input = Input(shape=(max_seq_len,20))
    # text-CNN
#     卷积池化-全连接 ---拼接-全连接  -双向GRU-全连接
    cnn = Conv1D(128, 3)(X_input)
#     cnn = BatchNormalization()(cnn)
    cnn = Activation('relu')(cnn)
     
    cnn = GlobalAveragePooling1D()(cnn)
    
    
    model = Model(X_input, cnn)
    return model


def build_siamese_model():
    left_input_seq = Input(shape=(max_seq_len,20))  
    right_input_seq = Input(shape=(max_seq_len,20))
    
    siamese_a = create_share_model()
    siamese_b = create_share_model()
    

    encoded_l = siamese_a(left_input_seq)
    encoded_r = siamese_b(right_input_seq)
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
#     dense = Dense(128,activation='relu')(L1_distance)
    prediction = Dense(1,activation='sigmoid')(L1_distance)

    siamese_net = Model(inputs=[left_input_seq,right_input_seq],outputs=prediction)
    
    siamese_net.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return siamese_net

def build_model():
    con_filters = 128
    
    left_input_seq = Input(shape=(max_seq_len,20))  
    right_input_seq = Input(shape=(max_seq_len,20))
    
    x = inception_block(left_input_seq,con_filters )
    x = Dropout(0.3)(x)
    x_gru = Bidirectional(CuDNNGRU(64, return_sequences=True))(left_input_seq)
    x_gru = Dropout(0.3)(x_gru)
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_gru_a = GlobalAveragePooling1D()(x_gru)
    x_gru_b = GlobalMaxPooling1D()(x_gru)
    left_x_seq = Concatenate()([x_a  , x_b, x_gru_a, x_gru_b])
    left_x_seq = Dense(256, activation='relu')(left_x_seq)
     
 
    x = inception_block(right_input_seq,con_filters )
    x = Dropout(0.3)(x)
    x_gru =Bidirectional(CuDNNGRU(64, return_sequences=True))(right_input_seq)
    x_gru = Dropout(0.3)(x_gru)
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_gru_a = GlobalAveragePooling1D()(x_gru)
    x_gru_b = GlobalMaxPooling1D()(x_gru)  
    right_x_seq = Concatenate()([x_a  , x_b, x_gru_a, x_gru_b])
    right_x_seq = Dense(256, activation='relu')(right_x_seq)
     
    x =   Concatenate()([left_x_seq, right_x_seq])
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.1)(x)
  
    x = Dense(1)(x)
    output = Activation('sigmoid')(x)
    # model = Model([left_input_go, right_input_go], output)
  
    model = Model([left_input_seq, right_input_seq], output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = build_model()
model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000, 20)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1000, 20)     0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 1000, 32)     1952        input_1[0][0]                    
___________________________________________________________________________

In [6]:
from sklearn.model_selection import StratifiedKFold
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
dataset_name = 'DM2'
for rep in range(3):
    n_splits = 10
    TPRs =  np.zeros(n_splits)
    FPRs = np.zeros(n_splits)
    Precs = np.zeros(n_splits)
    ACCs = np.zeros(n_splits)
    F1s = np.zeros(n_splits)
    MCCs = np.zeros(n_splits)
    AUCs = np.zeros(n_splits)
    count = 0
    for split in range(n_splits):
        train_pairs_file = 'CV/train'+str(rep)+'-'+str(split)
        test_pairs_file = 'CV/test'+str(rep)+'-'+str(split)
        valid_pairs_file = 'CV/valid'+str(rep)+'-'+str(split)

        batch_size = 32
        train_generator = DataGenerator(   train_pairs_file,batch_size = batch_size )
        test_generator = DataGenerator(   test_pairs_file,batch_size = batch_size)
        valid_generator = DataGenerator(   valid_pairs_file,batch_size = batch_size)
         

        model = build_model()
        save_model_name = 'CV/seq'+str(rep)+'-'+str(split) + '.hdf5'
        
        earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
        save_checkpoint = ModelCheckpoint(save_model_name, save_best_only=True, monitor='val_loss', mode='min', save_weights_only=True)

         
        # validation_data = (valid_X, valid_Y),  verbose=1,callbacks=[earlyStopping, save_checkpoint]
        hist = model.fit_generator(generator=train_generator,
                    validation_data=valid_generator,
                    epochs = 100,verbose=1,callbacks=[earlyStopping, save_checkpoint] )
         
        
        # model = load_model(save_model_name)
        model.load_weights(save_model_name)
        with open(test_pairs_file, 'r') as f:
            test_ppi_pairs  =  f.readlines()

        test_len = len(test_ppi_pairs) 
        list_IDs_temp = np.arange(test_len)

        test_x, y_test = test_generator.all_data(list_IDs_temp)

        y_pred_prob = model.predict(test_x)

       
        y_pred = (y_pred_prob > 0.5)
        auc = metrics.roc_auc_score(y_test, y_pred_prob) 
        f1 = f1_score(y_test, y_pred)
        pre = precision_score(y_test, y_pred)
        acc = accuracy_score(y_test, y_pred)

        precision, recall, _thresholds = metrics.precision_recall_curve(y_test, y_pred_prob)
        pr_auc = metrics.auc(recall, precision)
        mcc = matthews_corrcoef(y_test, y_pred)

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        total=tn+fp+fn+tp
        sen = float(tp)/float(tp+fn)
        sps = float(tn)/float((tn+fp))

        tpr = float(tp)/float(tp+fn)
        fpr = float(fp)/float((tn+fp))
        print('--------------------------\n')
        print ('AUC: %f' % auc)
        print ('ACC: %f' % acc) 
        # print("PRAUC: %f" % pr_auc)
        print ('MCC : %f' % mcc)
        # print ('SEN: %f' % sen)
        # print ('SEP: %f' % sps)
        print('TPR:%f'%tpr)
        print('FPR:%f'%fpr)
        print('Pre:%f'%pre)
        print('F1:%f'%f1)
        print('--------------------------\n')
        TPRs[count] = tpr
        FPRs[count] = fpr
        Precs[count] =pre
        ACCs[count] =acc
        F1s[count] =f1
        MCCs[count] =mcc
        AUCs[count] =auc
        count += 1
    print ('mean AUC: %f' % np.mean(AUCs))
    print ('mean ACC: %f' % np.mean(ACCs)) 
    print ('mean MCC : %f' % np.mean(MCCs))
    print('mean TPR:%f'% np.mean(TPRs))
    print('mean FPR:%f'% np.mean(FPRs))
    print('mean Pre:%f'% np.mean(Precs))
    print('mean F1:%f'% np.mean(F1s))
    np.savez('sp_seq_'+str(rep), AUCs=AUCs, ACCs=ACCs, MCCs=MCCs, TPRs = TPRs, FPRs=FPRs, Precs=Precs, F1s=F1s)

Epoch 1/100
33/33 [==============================] - 12s 355ms/step - loss: 0.7060 - acc: 0.5161 - val_loss: 0.6933 - val_acc: 0.4883
Epoch 2/100
33/33 [==============================] - 6s 176ms/step - loss: 0.6945 - acc: 0.4848 - val_loss: 0.6933 - val_acc: 0.4883
Epoch 3/100
33/33 [==============================] - 7s 226ms/step - loss: 0.6932 - acc: 0.5152 - val_loss: 0.6919 - val_acc: 0.5234
Epoch 4/100
33/33 [==============================] - 8s 244ms/step - loss: 0.6871 - acc: 0.5417 - val_loss: 0.6893 - val_acc: 0.5195
Epoch 5/100
33/33 [==============================] - 8s 244ms/step - loss: 0.6691 - acc: 0.5852 - val_loss: 0.6899 - val_acc: 0.5195
Epoch 6/100
33/33 [==============================] - 8s 244ms/step - loss: 0.5784 - acc: 0.6780 - val_loss: 0.6326 - val_acc: 0.6562
Epoch 7/100
33/33 [==============================] - 8s 246ms/step - loss: 0.5281 - acc: 0.7453 - val_loss: 0.5965 - val_acc: 0.6641
Epoch 8/100
33/33 [==============================] - 8s 238ms/step -

33/33 [==============================] - 8s 244ms/step - loss: 0.1233 - acc: 0.9593 - val_loss: 1.0285 - val_acc: 0.5898
Epoch 19/100
33/33 [==============================] - 8s 245ms/step - loss: 0.2372 - acc: 0.9034 - val_loss: 0.9251 - val_acc: 0.6016
Epoch 20/100
33/33 [==============================] - 8s 242ms/step - loss: 0.1970 - acc: 0.9148 - val_loss: 0.8662 - val_acc: 0.6094
Epoch 21/100
33/33 [==============================] - 8s 238ms/step - loss: 0.1043 - acc: 0.9697 - val_loss: 0.8819 - val_acc: 0.6641
Epoch 22/100
33/33 [==============================] - 8s 242ms/step - loss: 0.0909 - acc: 0.9659 - val_loss: 0.9008 - val_acc: 0.6758
--------------------------

AUC: 0.938840
ACC: 0.858108
MCC : 0.713351
TPR:0.848485
FPR:0.134146
Pre:0.835821
F1:0.842105
--------------------------

Epoch 1/100
33/33 [==============================] - 12s 349ms/step - loss: 0.7075 - acc: 0.4943 - val_loss: 0.6910 - val_acc: 0.5703
Epoch 2/100
33/33 [==============================] - 5s 165

33/33 [==============================] - 5s 167ms/step - loss: 0.1227 - acc: 0.9517 - val_loss: 0.6426 - val_acc: 0.7734
Epoch 20/100
33/33 [==============================] - 6s 173ms/step - loss: 0.1171 - acc: 0.9489 - val_loss: 0.6538 - val_acc: 0.7656
Epoch 21/100
33/33 [==============================] - 7s 204ms/step - loss: 0.1444 - acc: 0.9432 - val_loss: 0.6944 - val_acc: 0.7305
Epoch 22/100
33/33 [==============================] - 5s 164ms/step - loss: 0.1279 - acc: 0.9508 - val_loss: 0.8132 - val_acc: 0.6875
Epoch 23/100
33/33 [==============================] - 6s 174ms/step - loss: 0.1108 - acc: 0.9489 - val_loss: 0.7101 - val_acc: 0.7383
--------------------------

AUC: 0.936797
ACC: 0.837838
MCC : 0.690805
TPR:0.765432
FPR:0.074627
Pre:0.925373
F1:0.837838
--------------------------

Epoch 1/100
33/33 [==============================] - 15s 441ms/step - loss: 0.7125 - acc: 0.5237 - val_loss: 0.7000 - val_acc: 0.4688
Epoch 2/100
33/33 [==============================] - 8s 248

33/33 [==============================] - 6s 187ms/step - loss: 0.1567 - acc: 0.9384 - val_loss: 0.8017 - val_acc: 0.6797
Epoch 16/100
33/33 [==============================] - 6s 186ms/step - loss: 0.1432 - acc: 0.9451 - val_loss: 0.9145 - val_acc: 0.6562
Epoch 17/100
33/33 [==============================] - 6s 171ms/step - loss: 0.1206 - acc: 0.9479 - val_loss: 0.7590 - val_acc: 0.6953
Epoch 18/100
33/33 [==============================] - 6s 172ms/step - loss: 0.2321 - acc: 0.9072 - val_loss: 0.7473 - val_acc: 0.6719
--------------------------

AUC: 0.934432
ACC: 0.871622
MCC : 0.744557
TPR:0.923077
FPR:0.185714
Pre:0.847059
F1:0.883436
--------------------------

Epoch 1/100
33/33 [==============================] - 15s 469ms/step - loss: 0.7130 - acc: 0.4972 - val_loss: 0.6920 - val_acc: 0.5273
Epoch 2/100
33/33 [==============================] - 8s 246ms/step - loss: 0.6954 - acc: 0.5227 - val_loss: 0.6923 - val_acc: 0.5273
Epoch 3/100
33/33 [==============================] - 8s 243m

33/33 [==============================] - 5s 167ms/step - loss: 0.2090 - acc: 0.9167 - val_loss: 0.6594 - val_acc: 0.7070
Epoch 15/100
33/33 [==============================] - 6s 175ms/step - loss: 0.2122 - acc: 0.9100 - val_loss: 0.6219 - val_acc: 0.7383
Epoch 16/100
33/33 [==============================] - 6s 177ms/step - loss: 0.1667 - acc: 0.9394 - val_loss: 0.6488 - val_acc: 0.7227
Epoch 17/100
33/33 [==============================] - 7s 202ms/step - loss: 0.1262 - acc: 0.9489 - val_loss: 0.7005 - val_acc: 0.7109
Epoch 18/100
33/33 [==============================] - 6s 168ms/step - loss: 0.2115 - acc: 0.9138 - val_loss: 1.0410 - val_acc: 0.6484
--------------------------

AUC: 0.915673
ACC: 0.833333
MCC : 0.667327
TPR:0.794521
FPR:0.129870
Pre:0.852941
F1:0.822695
--------------------------

Epoch 1/100
33/33 [==============================] - 17s 513ms/step - loss: 0.7172 - acc: 0.5114 - val_loss: 0.6947 - val_acc: 0.4766
Epoch 2/100
33/33 [==============================] - 8s 245

...

Epoch 17/100
33/33 [==============================] - 6s 176ms/step - loss: 0.3691 - acc: 0.8390 - val_loss: 0.7087 - val_acc: 0.5898
Epoch 18/100
33/33 [==============================] - 6s 175ms/step - loss: 0.3511 - acc: 0.8551 - val_loss: 0.7588 - val_acc: 0.5781
Epoch 19/100
33/33 [==============================] - 6s 174ms/step - loss: 0.3457 - acc: 0.8523 - val_loss: 0.7235 - val_acc: 0.5859
Epoch 20/100
33/33 [==============================] - 6s 174ms/step - loss: 0.3498 - acc: 0.8381 - val_loss: 0.8635 - val_acc: 0.6016
Epoch 21/100
33/33 [==============================] - 6s 172ms/step - loss: 0.3557 - acc: 0.8438 - val_loss: 0.7568 - val_acc: 0.6523
Epoch 22/100
33/33 [==============================] - 6s 172ms/step - loss: 0.3235 - acc: 0.8608 - val_loss: 0.7448 - val_acc: 0.6055
--------------------------

AUC: 0.875112
ACC: 0.793333
MCC : 0.589668
TPR:0.666667
FPR:0.098765
Pre:0.851852
F1:0.747967
--------------------------

Epoch 1/100
33/33 [===========================

Epoch 2/100
33/33 [==============================] - 5s 167ms/step - loss: 0.6942 - acc: 0.5246 - val_loss: 0.6975 - val_acc: 0.4531
Epoch 3/100
33/33 [==============================] - 6s 177ms/step - loss: 0.6944 - acc: 0.5161 - val_loss: 0.6938 - val_acc: 0.4531
Epoch 4/100
33/33 [==============================] - 6s 175ms/step - loss: 0.6942 - acc: 0.4943 - val_loss: 0.6963 - val_acc: 0.4531
Epoch 5/100
33/33 [==============================] - 6s 177ms/step - loss: 0.6936 - acc: 0.5047 - val_loss: 0.6934 - val_acc: 0.4531
Epoch 6/100
33/33 [==============================] - 6s 174ms/step - loss: 0.6913 - acc: 0.5284 - val_loss: 0.7042 - val_acc: 0.4531
Epoch 7/100
33/33 [==============================] - 6s 175ms/step - loss: 0.6909 - acc: 0.5369 - val_loss: 0.6922 - val_acc: 0.4688
Epoch 8/100
33/33 [==============================] - 6s 180ms/step - loss: 0.6809 - acc: 0.5720 - val_loss: 0.6786 - val_acc: 0.6289
Epoch 9/100
33/33 [==============================] - 6s 176ms/step - 

33/33 [==============================] - 8s 250ms/step - loss: 0.6714 - acc: 0.6259 - val_loss: 0.6694 - val_acc: 0.5703
Epoch 8/100
33/33 [==============================] - 8s 249ms/step - loss: 0.5699 - acc: 0.7131 - val_loss: 0.6144 - val_acc: 0.6328
Epoch 9/100
33/33 [==============================] - 8s 247ms/step - loss: 0.4932 - acc: 0.7680 - val_loss: 0.6453 - val_acc: 0.6055
Epoch 10/100
33/33 [==============================] - 8s 245ms/step - loss: 0.4446 - acc: 0.7822 - val_loss: 0.5708 - val_acc: 0.7031
Epoch 11/100
33/33 [==============================] - 8s 246ms/step - loss: 0.4074 - acc: 0.8134 - val_loss: 0.6993 - val_acc: 0.6055
Epoch 12/100
33/33 [==============================] - 8s 252ms/step - loss: 0.3213 - acc: 0.8684 - val_loss: 0.5699 - val_acc: 0.7227
Epoch 13/100
33/33 [==============================] - 7s 209ms/step - loss: 0.2970 - acc: 0.8778 - val_loss: 0.7094 - val_acc: 0.6133
Epoch 14/100
33/33 [==============================] - 6s 184ms/step - loss: 0

33/33 [==============================] - 6s 178ms/step - loss: 0.6933 - acc: 0.5076 - val_loss: 0.6936 - val_acc: 0.4766
Epoch 8/100
33/33 [==============================] - 6s 180ms/step - loss: 0.6935 - acc: 0.5076 - val_loss: 0.6941 - val_acc: 0.4766
Epoch 9/100
33/33 [==============================] - 6s 170ms/step - loss: 0.6935 - acc: 0.4886 - val_loss: 0.6941 - val_acc: 0.4766
Epoch 10/100
33/33 [==============================] - 6s 178ms/step - loss: 0.6940 - acc: 0.5208 - val_loss: 0.6940 - val_acc: 0.4766
Epoch 11/100
33/33 [==============================] - 6s 178ms/step - loss: 0.6932 - acc: 0.5076 - val_loss: 0.6940 - val_acc: 0.4766
Epoch 12/100
33/33 [==============================] - 7s 218ms/step - loss: 0.6931 - acc: 0.5076 - val_loss: 0.6943 - val_acc: 0.4766
Epoch 13/100
33/33 [==============================] - 6s 179ms/step - loss: 0.6931 - acc: 0.5076 - val_loss: 0.6942 - val_acc: 0.4766
--------------------------

AUC: 0.715749
ACC: 0.479730
MCC : 0.000000
TPR:1.

/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [7]:
import numpy as np 
results1 =   np.load( 'sp_seq_0.npz')
results2 =   np.load( 'sp_seq_1.npz')
results3 =   np.load( 'sp_seq_2.npz')
print ('mean AUC: %f' %  ( (np.mean( results1[ 'AUCs' ] )  + np.mean(  results2[ 'AUCs' ] )  + np.mean(results3[ 'AUCs' ]))/3     ) )
print ('mean ACC: %f' %   ( (np.mean( results1[ 'ACCs' ] )  + np.mean(  results2[ 'ACCs' ] )  + np.mean(results3[ 'ACCs' ]))/3) )
print ('mean MCC : %f' %  (  (np.mean( results1[ 'MCCs' ] )  + np.mean(  results2[ 'MCCs' ] )  + np.mean(results3[ 'MCCs' ])     )/3))
print('mean TPR:%f'%    ((np.mean( results1[ 'TPRs' ] )  + np.mean(  results2[ 'TPRs' ] )  + np.mean(results3[ 'TPRs' ])     )/3))
print('mean FPR:%f'%   ( (np.mean( results1[ 'FPRs' ] )  + np.mean(  results2[ 'FPRs' ] )  + np.mean(results3[ 'FPRs' ])     )/3))
print('mean Pre:%f'%    ((np.mean( results1[ 'Precs' ] )  + np.mean(  results2[ 'Precs' ] )  + np.mean(results3[ 'Precs' ])     )/3))
print('mean F1:%f'%    ((np.mean( results1[ 'F1s' ] )  + np.mean(  results2[ 'F1s' ] )  + np.mean(results3[ 'F1s' ])     )/3))

mean AUC: 0.876276
mean ACC: 0.781087
mean MCC : 0.570536
mean TPR:0.831707
mean FPR:0.267178
mean Pre:0.811130
mean F1:0.800950
